<a href="https://colab.research.google.com/github/Sanatan-Shrivastava/hacksmu/blob/main/anomaly_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV


In [105]:
data = pd.read_excel('/content/HVAC Data (1).xlsx')

In [106]:
data.head(10)

,Supply Air Temperature,Static Pressure,Differential Pressure,Airflow Rate,Water Flow Rate (for hydronic systems),Relative Humidity,Zone Temperature,Carbon Dioxide (CO2) Concentration,Fan Speed,Temperature Setpoints,Humidity Setpoints,HVAC Energy Consumption,Outdoor Temperature,Outdoor Humidity,HVAC Operation Modes,Return Air Temperature
0,73°F,0.6 inches of water,0.06 inches of water,1100 cubic feet per minute,6 gallons per minute,0.55,71°F,410 ppm,1900 RPM,73°F,0.55,11 kWh,76°F,0.55,Cooling,69°F
1,74°F,0.7 inches of water,0.07 inches of water,1200 cubic feet per minute,7 gallons per minute,0.60,72°F,420 ppm,2000 RPM,74°F,0.60,12 kWh,77°F,0.60,Cooling,70°F
2,75°F,0.8 inches of water,0.08 inches of water,1300 cubic feet per minute,8 gallons per minute,0.65,73°F,430 ppm,2100 RPM,75°F,0.65,13 kWh,78°F,0.65,Cooling,71°F
3,76°F,0.9 inches of water,0.09 inches of water,1400 cubic feet per minute,9 gallons per minute,0.70,74°F,440 ppm,2200 RPM,76°F,0.70,14 kWh,79°F,0.70,Cooling,72°F
4,77°F,1.0 inches of water,0.10 inches of water,1500 cubic feet per minute,10 gallons per minute,0.75,75°F,450 ppm,2300 RPM,77°F,0.75,15 kWh,80°F,0.75,Cooling,73°F
5,82°F,1.6 inches of water,0.16 inches of water,2000 cubic feet per minute,12 gallons per minute,0.80,80°F,500 ppm,2800 RPM,82°F,0.80,20 kWh,85°F,NaN,NaN,78°F
6,83°F,1.7 inches of water,0.17 inches of water,2100 cubic feet per minute,13 gallons per minute,0.85,81°F,510 ppm,2900 RPM,83°F,0.85,21 kWh,86°F,0.80,Cooling,79°F
7,84°F,1.8 inches of water,0.18 inches of water,2200 cubic feet per minute,14 gallons per minute,0.90,82°F,520 ppm,3000 RPM,84°F,0.90,22 kWh,87°F,0.85,Cooling,80°F
8,78°F,1.1 inches of water,0.11 inches of water,1600 cubic feet per minute,11 gallons per minute,0.70,76°F,460 ppm,2400 RPM,78°F,0.70,16 kWh,81°F,0.70,Cooling,74°F
9,79°F,1.2 inches of water,0.12 inches of water,1700 cubic feet per minute,12 gallons per minute,0.75,77°F,470 ppm,2500 RPM,79°F,0.75,17 kWh,82°F,0.75,Cooling,75°F


In [107]:
data.columns

Index(['Supply Air Temperature', 'Static Pressure', 'Differential Pressure',
       'Airflow Rate', 'Water Flow Rate (for hydronic systems)',
       'Relative Humidity', 'Zone Temperature',
       'Carbon Dioxide (CO2) Concentration', 'Fan Speed',
       'Temperature Setpoints', 'Humidity Setpoints',
       'HVAC Energy Consumption', 'Outdoor Temperature', 'Outdoor Humidity',
       'HVAC Operation Modes', 'Return Air Temperature'],
      dtype='object')

In [108]:
data = data.drop(['Water Flow Rate (for hydronic systems)', 'HVAC Operation Modes'], axis =1)

In [109]:
data.columns

Index(['Supply Air Temperature', 'Static Pressure', 'Differential Pressure',
       'Airflow Rate', 'Relative Humidity', 'Zone Temperature',
       'Carbon Dioxide (CO2) Concentration', 'Fan Speed',
       'Temperature Setpoints', 'Humidity Setpoints',
       'HVAC Energy Consumption', 'Outdoor Temperature', 'Outdoor Humidity',
       'Return Air Temperature'],
      dtype='object')

In [110]:
# Specify the columns you want to process
columns_to_process = ['Supply Air Temperature',
                      'Static Pressure', 'Differential Pressure',
       'Airflow Rate','Carbon Dioxide (CO2) Concentration',
                      'Zone Temperature',
    'Fan Speed',
    'Temperature Setpoints',
    'HVAC Energy Consumption',
    'Outdoor Temperature',
    'Return Air Temperature'
]

# Use str.replace to remove text from specified columns
for col in columns_to_process:
    data[col] = data[col].str.replace(r'[a-zA-Z°]', '', regex=True)

# Print the updated DataFrame
data.head(10)

,Supply Air Temperature,Static Pressure,Differential Pressure,Airflow Rate,Relative Humidity,Zone Temperature,Carbon Dioxide (CO2) Concentration,Fan Speed,Temperature Setpoints,Humidity Setpoints,HVAC Energy Consumption,Outdoor Temperature,Outdoor Humidity,Return Air Temperature
0,73,0.6,0.06,1100,0.55,71,410,1900,73,0.55,11,76,0.55,69
1,74,0.7,0.07,1200,0.60,72,420,2000,74,0.60,12,77,0.60,70
2,75,0.8,0.08,1300,0.65,73,430,2100,75,0.65,13,78,0.65,71
3,76,0.9,0.09,1400,0.70,74,440,2200,76,0.70,14,79,0.70,72
4,77,1.0,0.10,1500,0.75,75,450,2300,77,0.75,15,80,0.75,73
5,82,1.6,0.16,2000,0.80,80,500,2800,82,0.80,20,85,NaN,78
6,83,1.7,0.17,2100,0.85,81,510,2900,83,0.85,21,86,0.80,79
7,84,1.8,0.18,2200,0.90,82,520,3000,84,0.90,22,87,0.85,80
8,78,1.1,0.11,1600,0.70,76,460,2400,78,0.70,16,81,0.70,74
9,79,1.2,0.12,1700,0.75,77,470,2500,79,0.75,17,82,0.75,75


In [111]:
data.fillna(data.median(), inplace=True)

In [112]:
data.isnull().sum()

Supply Air Temperature                0
Static Pressure                       0
Differential Pressure                 0
Airflow Rate                          0
Relative Humidity                     0
Zone Temperature                      0
Carbon Dioxide (CO2) Concentration    0
Fan Speed                             0
Temperature Setpoints                 0
Humidity Setpoints                    0
HVAC Energy Consumption               0
Outdoor Temperature                   0
Outdoor Humidity                      0
Return Air Temperature                0
dtype: int64

In [117]:

# Assuming you have preprocessed and scaled your data as mentioned earlier
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# Create the Isolation Forest model
model = IsolationForest(n_estimators=100, contamination=0.05, random_state=42)

# Fit the model to your data
model.fit(data_scaled)

# Predict anomalies
predictions = model.predict(data_scaled)


In [114]:
num_anomalies = (predictions == -1).sum()
print("Number of anomalies:", num_anomalies)

Number of anomalies: 62


In [115]:
data['Anomaly'] = predictions

In [116]:
data

,Supply Air Temperature,Static Pressure,Differential Pressure,Airflow Rate,Relative Humidity,Zone Temperature,Carbon Dioxide (CO2) Concentration,Fan Speed,Temperature Setpoints,Humidity Setpoints,HVAC Energy Consumption,Outdoor Temperature,Outdoor Humidity,Return Air Temperature,Anomaly
0,73,0.6,0.06,1100,0.55,71,410,1900,73,0.55,11,76,0.55,69,1
1,74,0.7,0.07,1200,0.60,72,420,2000,74,0.60,12,77,0.60,70,1
2,75,0.8,0.08,1300,0.65,73,430,2100,75,0.65,13,78,0.65,71,1
3,76,0.9,0.09,1400,0.70,74,440,2200,76,0.70,14,79,0.70,72,1
4,77,1.0,0.10,1500,0.75,75,450,2300,77,0.75,15,80,0.75,73,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233,78,0.6,0.15,1600,0.69,76,475,2300,78,0.69,17.0,81,0.69,75,1
1234,84,1.6,0.21,1900,0.75,82,505,2600,84,0.75,20.0,87,0.75,81,1
1235,85,1.7,0.22,1950,0.76,83,510,2650,85,0.76,20.5,88,0.76,82,1
1236,88,2.0,0.25,2100,0.79,86,525,2800,88,0.79,22.0,91,0.79,85,1


In [120]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt


# Create the outlier detection models
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
svm = OneClassSVM(nu=0.05)
dbscan = DBSCAN(eps=0.5, min_samples=5)

# Fit the models to your data and make predictions
lof_predictions = lof.fit_predict(data_scaled)
svm_predictions = svm.fit_predict(data_scaled)
dbscan_predictions = dbscan.fit_predict(data_scaled)

# 2. Silhouette Score (if clustering is possible)
silhouette_avg_lof = silhouette_score(data_scaled, lof_predictions)
silhouette_avg_svm = silhouette_score(data_scaled, svm_predictions)
silhouette_avg_dbscan = silhouette_score(data_scaled, dbscan_predictions)

print("Silhouette Score (LOF):", silhouette_avg_lof)
print("Silhouette Score (SVM):", silhouette_avg_svm)
print("Silhouette Score (DBSCAN):", silhouette_avg_dbscan)

# 3. Quantitative Measures (e.g., threshold-based)
threshold = -1  # Adjust the threshold as needed

lof_anomalies = data[lof_predictions == -1]
svm_anomalies = data[svm_predictions == -1]
dbscan_anomalies = data[dbscan_predictions == -1]


print("Number of anomalies (LOF):", len(lof_anomalies))
print("Number of anomalies (SVM):", len(svm_anomalies))
print("Number of anomalies (DBSCAN):", len(dbscan_anomalies))



Silhouette Score (LOF): 0.2753472764774295
Silhouette Score (SVM): 0.2463113957694119
Silhouette Score (DBSCAN): 0.8480974535327508
Number of anomalies (LOF): 56
Number of anomalies (SVM): 134
Number of anomalies (DBSCAN): 52
